## Prepare Data for ViT-based estimation method

used code in ViT-pytorch/

In [1]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path
import os, sys
import numpy as np
import torch

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.visualizer import COCOVisualizer
from util import box_ops
import pickle
import copy
import random
from util.utils import slprint, to_device
import util.misc as utils
from engine import evaluate
from torch.utils.data import DataLoader
from datasets import build_dataset, get_coco_api_from_dataset

In [91]:
split = "val"
base_path = "./data/5_scale_31/"
data_path = base_path + split + "/data/"
annotation_path = base_path + split + "/box_annotation/"

In [84]:
def read_one_image_results(path):
    with open(path, "r") as outfile:
        data = json.load(outfile)
    return data

def write_one_results(path, json_data):
    with open(path, "w") as outfile:
        json.dump(json_data, outfile)
        
def get_numpy_data(data_path, annotation_path, img_nums):
    X = None
    Y = None
    for img_idx in range(img_nums):
        results = read_one_image_results(data_path + str(img_idx) + ".json")
        pred_logits = np.array(results['input']['pred_logits'])
        pred_boxes = np.array(results['input']['pred_boxes'])
        pred_results = np.concatenate((pred_boxes, pred_logits), axis=2)
        annotation_data = read_one_image_results(annotation_path + str(img_idx) + ".json")
        selected_index = annotation_data['selected_index']
        out_results = pred_results[:,selected_index]
        loss = annotation_data['loss']
        pred_logits_max = np.max(pred_logits, axis=2).squeeze()
        sort_indexs = np.argsort(-pred_logits_max)
        topk_indexs = sort_indexs[:196]
        pred_results = pred_results[:,topk_indexs]
        arrSortedIndex  = np.lexsort((pred_results[:,:,0], pred_results[:,:,1])).squeeze()
        pred_results = pred_results[:,arrSortedIndex]
        one_X = None
        for i in range(out_results.shape[1]):
            temp = np.append(out_results[:,i], pred_results)
            temp = temp.reshape((1,pred_results.shape[1]+1, pred_results.shape[2]))
            if one_X is None:
                one_X = temp
            else:
                one_X = np.concatenate((one_X, temp), axis=0)
        if X is None:
            X = one_X
        else:
            X = np.concatenate((X, one_X), axis=0)
        if Y is None:
            Y = loss
        else:
            Y = np.concatenate((Y, loss))
        if img_idx % 100 == 0:
            print(f"{img_idx} finished")
    return X, Y

In [85]:
train_data_path = "./data/5_scale_31/train/data/"
test_data_path = "./data/5_scale_31/val/data/"
train_annotation_path = "./data/5_scale_31/train/box_annotation/"
test_annotation_path = "./data/5_scale_31/val/box_annotation/"

In [ ]:
test_X, test_Y = get_numpy_data(test_data_path, test_annotation_path, 5000)

0 finished
100 finished
200 finished
300 finished
400 finished
500 finished
600 finished
700 finished
800 finished
900 finished
1000 finished
1100 finished
1200 finished
1300 finished
1400 finished
1500 finished
1600 finished
1700 finished
1800 finished
1900 finished
2000 finished
2100 finished
2200 finished
2300 finished
2400 finished
2500 finished
2600 finished
2700 finished
2800 finished
2900 finished
3000 finished
3100 finished
3200 finished
3300 finished
3400 finished
3500 finished
3600 finished
3700 finished
3800 finished
3900 finished


In [92]:
split = "val"
store_preprocess_inputs_path = base_path + split + f"/pre_data/{split}_box_level_ViT_inputs.npy"
with open(store_preprocess_inputs_path, "wb") as outfile:
    np.save(outfile, test_X)
store_preprocess_annotations_path = base_path + split + f"/pre_data/{split}_box_level_ViT_annotations.npy"
with open(store_preprocess_annotations_path, "wb") as outfile:
    np.save(outfile, test_Y)

In [94]:
test_X.shape, test_Y.shape

((49129, 197, 95), (49129,))

In [ ]:
train_X, train_Y = get_numpy_data(train_data_path, train_annotation_path, 50000)

0 finished
100 finished
200 finished
300 finished
400 finished
500 finished
600 finished
700 finished
800 finished
900 finished
1000 finished
1100 finished
1200 finished
1300 finished
1400 finished
1500 finished
1600 finished
1700 finished
1800 finished
1900 finished
2000 finished
2100 finished
2200 finished
2300 finished
2400 finished
2500 finished
2600 finished
2700 finished
2800 finished
2900 finished
3000 finished
3100 finished
3200 finished
3300 finished
3400 finished
3500 finished
3600 finished
3700 finished
3800 finished
3900 finished
4000 finished
4100 finished
4200 finished
4300 finished
4400 finished
4500 finished
4600 finished
4700 finished
4800 finished
4900 finished
5000 finished
5100 finished
5200 finished
5300 finished
5400 finished
5500 finished
5600 finished
5700 finished
5800 finished
5900 finished
6000 finished
6100 finished
6200 finished
6300 finished
6400 finished
6500 finished
6600 finished
6700 finished
6800 finished
6900 finished
7000 finished
7100 finished
7200

In [ ]:
split = "train"
store_preprocess_inputs_path = base_path + split + f"/pre_data/{split}_box_level_ViT_inputs.npy"
with open(store_preprocess_inputs_path, "wb") as outfile:
    np.save(outfile, train_X)
store_preprocess_annotations_path = base_path + split + f"/pre_data/{split}_box_level_ViT_annotations.npy"
with open(store_preprocess_annotations_path, "wb") as outfile:
    np.save(outfile, train_Y)